In [ ]:
# ==============================================================================
# SECTION 1: LIBRARY IMPORTS
# ==============================================================================
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import os # Import os for path joining
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    MaxPooling1D,
    Dropout,
    Dense,
    LayerNormalization,
    GlobalAveragePooling1D,
    SeparableConv1D 
)
from tensorflow.keras.metrics import Recall, Precision

# Ensure reproducibility
tf.random.set_seed(42)
np.random.seed(42)


# ==============================================================================
# SECTION 2: DATA LOADING AND PREPROCESSING (CRITICAL PATH FIX)
# ==============================================================================

# 🛑 CRITICAL FIX: Update this path to match your local file structure 🛑
# Example: If your .txt files are in './data/mote_readings/', set it to that.
# The files are likely in a folder named 'SingleHopLabelledReadings' after unzipping.
PATH_TO_DATA_FOLDER = '' 

# Base file names (the names of the .txt files)
base_file_names = [
    'singlehop-indoor-moteid1-data.txt',
    'singlehop-indoor-moteid2-data.txt',
    'singlehop-outdoor-moteid3-data.txt',
    'singlehop-outdoor-moteid4-data.txt'
]

# Use os.path.join to create the correct, accessible file paths
file_paths = [os.path.join(PATH_TO_DATA_FOLDER, name) for name in base_file_names]

COLUMNS = ['Reading_Num', 'Mote_ID', 'Humidity', 'Temperature', 'Label']
all_mote_data = []

def load_and_clean_mote_data(file_path, skip_rows):
    """
    Loads and cleans a single mote data file by skipping junk rows 
    and manually assigning the known columns.
    """
    try:
        # 1. Load data
        # sep=r'\s+' handles both tabs and multiple spaces as delimiters
        df = pd.read_csv(
            file_path, 
            sep=r'\s+',              
            header=None,             
            skiprows=skip_rows,      
            engine='python',         
            usecols=range(5),        
            on_bad_lines='skip',
            encoding='utf-8' 
        )
        
        # 2. MANUALLY ASSIGN THE CORRECT COLUMN NAMES (Ensures 'Label' exists)
        df.columns = COLUMNS
        
        # 3. Robust type conversion and cleaning
        for col in ['Reading_Num', 'Mote_ID', 'Label']:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
        
        for col in ['Humidity', 'Temperature']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # 4. Drop rows where critical data is missing
        df.dropna(subset=['Mote_ID', 'Humidity', 'Temperature', 'Label'], inplace=True)
        
        print(f"Loaded {len(df)} rows from {os.path.basename(file_path)}")
        return df.astype({'Mote_ID': int, 'Label': int}) 
    
    except FileNotFoundError:
        # Re-raising a clear error for the user to see the path issue
        print(f"Error processing {file_path}: [Errno 2] File Not Found. Check if PATH_TO_DATA_FOLDER is correct.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error processing {file_path}. Data not included: {e}")
        return pd.DataFrame()

# Mote-specific loading using the required skiprows for this dataset:
# We iterate through the base_file_names and apply the correct skip rows
skip_rules = {
    'singlehop-indoor-moteid1-data.txt': 4,
    'singlehop-indoor-moteid2-data.txt': 1,
    'singlehop-outdoor-moteid3-data.txt': 1,
    'singlehop-outdoor-moteid4-data.txt': 2
}

for file_name in base_file_names:
    full_path = os.path.join(PATH_TO_DATA_FOLDER, file_name)
    all_mote_data.append(load_and_clean_mote_data(full_path, skip_rules[file_name]))

# Concatenate all DataFrames
merged_df = pd.concat(all_mote_data, ignore_index=True)

# The rest of the code should now run successfully
# ... (Sequence creation, model building, training, and evaluation) ...
# ... (Continue with the code from the previous response from here) ...

print(f"\nTotal merged data points: {len(merged_df)}")
if len(merged_df) > 0:
    print(f"Fault distribution:\n{merged_df['Label'].value_counts(normalize=True).mul(100).round(2)}")

Error processing ./SingleHopLabelledReadings/SingleHopLabelledReadings/singlehop-indoor-moteid1-data.txt: [Errno 2] File Not Found. Check if PATH_TO_DATA_FOLDER is correct.
Error processing ./SingleHopLabelledReadings/SingleHopLabelledReadings/singlehop-indoor-moteid2-data.txt: [Errno 2] File Not Found. Check if PATH_TO_DATA_FOLDER is correct.
Error processing ./SingleHopLabelledReadings/SingleHopLabelledReadings/singlehop-outdoor-moteid3-data.txt: [Errno 2] File Not Found. Check if PATH_TO_DATA_FOLDER is correct.
Error processing ./SingleHopLabelledReadings/SingleHopLabelledReadings/singlehop-outdoor-moteid4-data.txt: [Errno 2] File Not Found. Check if PATH_TO_DATA_FOLDER is correct.

Total merged data points: 0
